# Variables

In [8]:

# reading the JSON document - FabricMonitoring_Variables.json
df = spark.read.option("multiline", "true").json("Files/FabricMonitoring_Variables.json")
tenantId = df.select("*").first()['tenantId']
clientId = df.first()['clientId']
key_vault_name = df.first()['nameOfTheKeyVAult']
key_name = df.first()['nameOfTheKey']
filepathTenantSettings = df.first()['filepathTenantSettings']


StatementMeta(, 73788b46-fb05-49b7-9309-72bc5470f463, 12, Finished, Available)

# All the imports

In [3]:
spark.conf.set("spark.sql.parquet.vorder.enabled", "true")

from azure.identity import ClientSecretCredential

# all the stuff I need
from notebookutils import mssparkutils
from pyspark.sql.types import StringType
from pyspark.sql.functions import lit
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
from pyspark.sql.functions import concat
from pyspark.sql.functions import get_json_object
from pyspark.sql.functions import from_json

import json
import requests
import azure.identity as ai
from datetime import datetime

StatementMeta(, 73788b46-fb05-49b7-9309-72bc5470f463, 7, Finished, Available)

# Prepare some of the authentication thingies

In [4]:
from trident_token_library_wrapper import PyTridentTokenLibrary as tl

access_token = mssparkutils.credentials.getToken("keyvault")

client_secret = tl.get_secret_with_token( \
    f"https://{key_vault_name}.vault.azure.net/", \
    key_name, \
    access_token
)

StatementMeta(, 73788b46-fb05-49b7-9309-72bc5470f463, 8, Finished, Available)

# Get the data

In [ ]:
# https://github.com/microsoft/azure-analysis-services-http-sample/blob/master/README.md
api = "https://analysis.windows.net/powerbi/api/.default"

# authenticating against the REST API endpoint using a SPN (client_id)
ClientSecretCredential_class = ai.ClientSecretCredential(tenant_id = tenantId, client_id = clientId, client_secret = client_secret)
AccessToken_class = ClientSecretCredential_class.get_token(api)
access_token = AccessToken_class.token
header = {'Authorization': f'Bearer {access_token}'}

TenantSettingsURL = 'https://api.fabric.microsoft.com/v1/admin/tenantsettings'
header = {'Authorization': f'Bearer {access_token}'}
TenantSettings_requestReturn = requests.get(TenantSettingsURL , headers=header)

TenantSettings_requestReturn.status_code

# when everything is fine
normalized_data = dict()
TenantSettings_json = TenantSettings_requestReturn.json()['tenantSettings']
TenantSettings_json_dumps = json.dumps(TenantSettings_json)
normalized_data["_data"] = TenantSettings_json_dumps
a = json.dumps( normalized_data["_data"] )
payload = json.loads(a)
payload_rdd = spark.sparkContext.parallelize([payload])

# create the dataframe
df_tenantSettings = spark.read \
    .option("inferSchema", "true") \
    .option("multiline","true") \
    .json(payload_rdd)

#display(df_tenantSettings)

In [ ]:
activity_DateTime = datetime.now()
activity_DateTime_date = activity_DateTime.strftime("%Y-%m-%d")

df_tenantSettings.write \
    .option("multiline", "true") \
    .mode("overwrite") \
    .json(f"{filepathTenantSettings}TenantSettings_{activity_DateTime_date}.json")

In [18]:
#print(df_tenantSettings.schema)

StatementMeta(, fc9c78bb-c421-43e1-9e51-75ef16c60f89, 22, Finished, Available)